In [1]:
%load_ext autoreload
%autoreload 1

import sys

sys.path.append("../utils/")

import os
import json
import pdb
from bs4 import BeautifulSoup
import time
import requests
import pandas as pd

from work_crawler import save_to_mlab

%aimport work_crawler

In [2]:
for root, dirs, files in os.walk("../json"):
    composers = [file.split(".")[0] for file in files]
    

In [3]:
composers = sorted(composers)
composers

['ascarlatti',
 'bartok',
 'beethoven',
 'bellini',
 'berg',
 'berlioz',
 'binchois',
 'bizet',
 'borodin',
 'brahms',
 'bruckner',
 'byrd',
 'caccini',
 'chopin',
 'corelli',
 'couperin',
 'cpebach',
 'debussy',
 'devitry',
 'donizetti',
 'dowland',
 'dscarlatti',
 'dufay',
 'dunstable',
 'dvorak',
 'elgar',
 'faure',
 'franck',
 'frescobaldi',
 'gabrieli',
 'gesualdo',
 'gibbons',
 'gluck',
 'gounod',
 'grieg',
 'handel',
 'haydn',
 'hindemith',
 'holst',
 'ives',
 'josquin',
 'jsbach',
 'lassus',
 'liszt',
 'lully',
 'machaut',
 'mahler',
 'mendelssohn',
 'meyerbeer',
 'monteverdi',
 'morley',
 'mozart',
 'mussorgsky',
 'ockeghem',
 'offenbach',
 'paganini',
 'palestrina',
 'pergolesi',
 'perotin',
 'praetorius',
 'prokofiev',
 'puccini',
 'purcell',
 'rachmaninoff',
 'rameau',
 'ravel',
 'rimskykorsakov',
 'rossini',
 'rstrauss',
 'saintsaens',
 'satie',
 'schoenberg',
 'schubert',
 'schumann',
 'schutz',
 'scriabin',
 'sibelius',
 'smetana',
 'stamitz',
 'stravinsky',
 'tallis',
 

In [5]:
def is_bytes(resp):
    try:
        resp.content.decode()
        return True
    except:
        return False

In [9]:
def json_range(composers, start=None, stop=None):
    start_index = 0
    stop_index = -1
    
    if start:
        start_index = composers.index(start)
        
    if stop:
        stop_index = composers.index(stop) + 1
        
    return composers[start_index:stop_index]

json_range(composers, start="rossini")
        

['rossini',
 'schubert',
 'dowland',
 'machaut',
 'rachmaninoff',
 'frescobaldi',
 'dscarlatti',
 'binchois',
 'weelkes',
 'ockeghem',
 'couperin',
 'webern',
 'smetana',
 'bellini',
 'puccini',
 'verdi',
 'dvorak',
 'liszt',
 'purcell',
 'wolf',
 'corelli',
 'dunstable',
 'varese',
 'debussy',
 'holst',
 'bartok',
 'ravel',
 'lully',
 'schutz',
 'sibelius',
 'caccini',
 'haydn',
 'rameau',
 'satie',
 'tchaikovsky',
 'borodin',
 'mahler',
 'rimskykorsakov',
 'mendelssohn',
 'chopin',
 'taverner',
 'mozart',
 'monteverdi',
 'stamitz',
 'perotin',
 'bruckner',
 'berlioz',
 'vivaldi',
 'schoenberg',
 'handel',
 'schumann',
 'saintsaens',
 'faure',
 'beethoven',
 'wagner',
 'palestrina',
 'grieg',
 'bizet',
 'brahms',
 'mussorgsky',
 'gibbons',
 'jsbach',
 'cpebach',
 'devitry',
 'pergolesi',
 'praetorius',
 'rstrauss',
 'meyerbeer',
 'offenbach',
 'gesualdo',
 'gabrieli',
 'ascarlatti',
 'donizetti',
 'stravinsky',
 'telemann',
 'vaughanwilliams',
 'franck',
 'lassus',
 'elgar',
 'tallis'

In [4]:
class ProcessComposer():
    
    def __init__(self, composer):
        composer_json = self._get_json(composer)
        self._create_composer_df(composer_json)
    
    def _get_json(self, composer):
        with open(f"../json/{composer}.json", "r") as file:
            composer_json = json.load(file)
        return composer_json
    
    def _create_composer_df(self, composer_json):
        composer_df = pd.DataFrame(composer_json)
        composer_df['score_ids'] = composer_df.scores.apply(lambda score_lst: [int(x.split("/")[-1]) for x in score_lst])
        self.composer_df = composer_df
       
    def _process_work(self, work):
        pdfs = []
        for score_id in work.score_ids:
            resp = requests.get(f"https://imslp.org/wiki/Special:IMSLPDisclaimerAccept/{score_id}", headers={"X-Forwarded-For": "24.18.100.31"})
            try:
#                 if is_bytes(resp):
#                     print("bytes")
#                     pdfs.append(resp.url)
#                 else:
#                 print("string")
                soup = BeautifulSoup(resp.content)
                pdfs.append(soup.find("span", {"id":"sm_dl_wait"}).get("data-id"))
            except:                
                time.sleep(2)
                continue
            
            time.sleep(2)
    
        return pdfs
    
    def _save_work(self, work_dict):
        save_to_mlab(work_dict)
    
    def run_work_retrieval(self, start=0, stop=-1):
        for i, row in self.composer_df[start:stop].iterrows():
            pdfs = self._process_work(row)
            row_dict = row.to_dict()
            
            row_dict["pdfs"] = pdfs
            print(row_dict)
            if len(pdfs) > 0:
                self._save_work(row_dict)
            

In [6]:
cp = ProcessComposer("jsbach")
# cp.composer_df


In [7]:
cp.run_work_retrieval(7, 8)

{'composer': 'Bach, Johann Sebastian', 'date': '1715', 'scores': ['https://imslp.org/wiki/Special:ImagefromIndex/01424'], 'tags': ['Sacred cantatas', 'Cantatas', 'Religious works', 'For 4 voices, mixed chorus, orchestra', 'For voices and chorus with orchestra'], 'title': 'Ach! ich sehe, itzt, da ich zur Hochzeit gehe, BWV 162', 'score_ids': [1424], 'pdfs': ['http://ks.imslp.net/files/imglnks/usimg/7/77/IMSLP01424-BWV0162.pdf']}
